# Chat API

* Connecting to Claude's API via [claudette](https://claudette.answer.ai/) for chat completion, 
* Extracting JSON and rules to populate gradio UI

In [2]:
#| default_exp chat

In [3]:
#| hide
from nbdev.showdoc import *

In [4]:
#| export
import claudette
from pathlib import Path
import re

* Claudette provides models, which is a list of models currently available from the SDK.

In [5]:
[print(m) for m in claudette.models]
# using Sonnet 3.5
model = claudette.models[3]
print(f'\nChosen model: {model}')

claude-3-opus-20240229
claude-3-7-sonnet-20250219
claude-3-5-sonnet-20241022
claude-3-haiku-20240307
claude-3-5-haiku-20241022

Chosen model: claude-3-haiku-20240307


In [6]:
import os
api_key = os.getenv('ANTHROPIC_API_KEY')
if api_key: print('key exists ☑')

key exists ☑


In [52]:

context_file = Path('docs/analytics.md')
context_media = context_file.read_text(encoding='utf-8')

system_prompt = """You are a helpful and concise assistant."""

chat = claudette.Chat(model, sp=system_prompt)
response = chat("""Can you come up with 10 realistic questions a hiring manager might ask in an opening interview?  the job title for this role is: SENIOR RESEARCH ANALYST,
            Do not add any additional commentary, just list 10 questions only. Start each question with a new line starting like:
                [Q1] ... 
                [Q2] ...
                My resume is in the project docs, the core competencies for a senior analyst are below:
                {context_media}""")


In [53]:
chat

[Q1] Can you tell me about your background and experience as a research analyst?

[Q2] What are your key strengths and how do they make you a good fit for this senior research analyst role?

[Q3] Can you describe a complex research project you've worked on and your specific contributions?

[Q4] How do you stay up-to-date with industry trends and developments relevant to this role?

[Q5] Can you provide an example of a time when you had to present your research findings to a client or stakeholder?

[Q6] How do you approach data analysis and interpretation to draw meaningful insights?

[Q7] What experience do you have in using data visualization tools to present research findings?

[Q8] Can you share a time when you had to navigate a challenging research-related problem or obstacle?

[Q9] How do you ensure the quality and accuracy of your research outputs?

[Q10] What are your long-term career goals and how does this senior research analyst role fit into your professional development?

<details>
<summary>History</summary>

**user**: C

**assistant**: [Q1] Can you tell me about your background and experience as a research analyst?

[Q2] What are your key strengths and how do they make you a good fit for this senior research analyst role?

[Q3] Can you describe a complex research project you've worked on and your specific contributions?

[Q4] How do you stay up-to-date with industry trends and developments relevant to this role?

[Q5] Can you provide an example of a time when you had to present your research findings to a client or stakeholder?

[Q6] How do you approach data analysis and interpretation to draw meaningful insights?

[Q7] What experience do you have in using data visualization tools to present research findings?

[Q8] Can you share a time when you had to navigate a challenging research-related problem or obstacle?

[Q9] How do you ensure the quality and accuracy of your research outputs?

[Q10] What are your long-term career goals and how does this senior research analyst role fit into your professional development?
</details>

| Metric | Count | Cost (USD) |
|--------|------:|-----:|
| Input tokens | 122 | 0.000030 |
| Output tokens | 228 | 0.000285 |
| Cache tokens | 0 | 0.000000 |
| **Total** | **350** | **$0.000316** |

## More directly testing on Mock Interviews:

In [20]:
#| export

INTERVIEW_INSTRUCTIONS = """Can you walk through each of the interview question from the. file, one by one, and I 
will reply directly to them in the response, as in a mock interview. I want you to set up the following sections 
with line breaks between them 

----- ANSWER EVAL ----- {Summarise user input answer concisely and into a few bullet points, (depending on length). 
This section will be displayed in Markdown, so format appropriately.
Use headings GOOD, with a small numbered list of what is considered high-quality response. 
Use heading BAD, critique of where the answer went wrong, think about verbosity, clarity, what kind of thing should be 
answered in a high level interview, for a very senior position like this.]

[RESPONSE METRICS] Here please return a json object with these metrics as keys and a score rating between 0-10 (1 d.p.). 
This is mostly for fun but do the best you can with it. The metrics:
* Linguistic Clarity
* Confidence
* Technical Expertise
* Strategic Thinking
* Impact Demonstration
* Industry Understanding
[ENDJSON]

----- REWRITEN ----- 
Use the above notes to write a first-in-class script for the question. Use the same general facts as the actual answer, 
but don't be afraid to add in external context to the answer if your descretion requires it. This is a fictional exercise 
and we want the answer to be an example of a perfect answer. Use bold to highly key words and phrases in the answer, and 
keep it conversational in tone. It needs to be spoken out loud, so avoid literary flourishes that would only exist on paper. 
----- END ----- 
If the users answers the question in the response the evaluate it like above, and then ask to move on to the next question. 
For each question and answer repeat the same instructions."""

In [21]:
chat(INTERVIEW_INSTRUCTIONS)

[Q1] Can you walk me through your relevant work experience as a research analyst?

<details>

- id: `msg_01S1gYrMsFaD5nouZySSMm28`
- content: `[{'citations': None, 'text': '[Q1] Can you walk me through your relevant work experience as a research analyst?', 'type': 'text'}]`
- model: `claude-3-haiku-20240307`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 755, 'output_tokens': 21}`

</details>

In [22]:
user_reply = "Yeah, when I was working as a research analyst, it was pretty tight. Lots of people asked me some pretty hard questions and I, you know, I just crushed it"
c = chat(user_reply)
c

----- ANSWER EVAL -----

**GOOD**:
1. Provided a high-level overview of relevant work experience as a research analyst.
2. Conveyed confidence in handling challenging situations.

**BAD**:
1. The response is quite vague and lacks specific details about the candidate's work experience.
2. The language used is informal and does not reflect the professionalism expected for a senior-level position.
3. The answer does not demonstrate the depth of knowledge and expertise required for a senior research analyst role.

[RESPONSE METRICS]
{
  "Linguistic Clarity": 3.0,
  "Confidence": 5.0,
  "Technical Expertise": 2.0,
  "Strategic Thinking": 2.0,
  "Impact Demonstration": 2.0,
  "Industry Understanding": 2.0
}

----- REWRITTEN -----

**As a senior research analyst, I have extensive experience in conducting in-depth market research, data analysis, and providing strategic insights to support key business decisions.** During my time at [Previous Employer], I led a team of analysts in a comprehensive study of the competitive landscape within our industry. This involved gathering and synthesizing data from multiple sources, including industry reports, customer surveys, and internal sales data. 

**I was able to identify emerging trends and pain points that our clients were facing, which allowed us to develop targeted solutions that delivered significant value.** For example, I spearheaded a project to analyze customer churn patterns, which led to the implementation of a proactive customer retention program that improved our client retention rate by 15% over the course of a year.

**Throughout my career, I have honed my skills in leveraging advanced data analysis tools and techniques to uncover meaningful insights that drive business growth.** I am proficient in statistical modeling, data visualization, and presenting complex findings to both technical and non-technical stakeholders in a clear and compelling manner. **My ability to translate data into actionable recommendations has been instrumental in helping my previous employers make informed, data-driven decisions.**

----- END -----

[Q2] How do you stay up-to-date with the latest industry trends and developments?

<details>

- id: `msg_01P7hvJ9Tr7kUUCVhzTwGiWL`
- content: `[{'citations': None, 'text': '----- ANSWER EVAL -----\n\n**GOOD**:\n1. Provided a high-level overview of relevant work experience as a research analyst.\n2. Conveyed confidence in handling challenging situations.\n\n**BAD**:\n1. The response is quite vague and lacks specific details about the candidate\'s work experience.\n2. The language used is informal and does not reflect the professionalism expected for a senior-level position.\n3. The answer does not demonstrate the depth of knowledge and expertise required for a senior research analyst role.\n\n[RESPONSE METRICS]\n{\n  "Linguistic Clarity": 3.0,\n  "Confidence": 5.0,\n  "Technical Expertise": 2.0,\n  "Strategic Thinking": 2.0,\n  "Impact Demonstration": 2.0,\n  "Industry Understanding": 2.0\n}\n\n----- REWRITTEN -----\n\n**As a senior research analyst, I have extensive experience in conducting in-depth market research, data analysis, and providing strategic insights to support key business decisions.** During my time at [Previous Employer], I led a team of analysts in a comprehensive study of the competitive landscape within our industry. This involved gathering and synthesizing data from multiple sources, including industry reports, customer surveys, and internal sales data. \n\n**I was able to identify emerging trends and pain points that our clients were facing, which allowed us to develop targeted solutions that delivered significant value.** For example, I spearheaded a project to analyze customer churn patterns, which led to the implementation of a proactive customer retention program that improved our client retention rate by 15% over the course of a year.\n\n**Throughout my career, I have honed my skills in leveraging advanced data analysis tools and techniques to uncover meaningful insights that drive business growth.** I am proficient in statistical modeling, data visualization, and presenting complex findings to both technical and non-technical stakeholders in a clear and compelling manner. **My ability to translate data into actionable recommendations has been instrumental in helping my previous employers make informed, data-driven decisions.**\n\n----- END -----\n\n[Q2] How do you stay up-to-date with the latest industry trends and developments?', 'type': 'text'}]`
- model: `claude-3-haiku-20240307`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 816, 'output_tokens': 488}`

</details>

# Test Mechanic – Inline `Gradio UI`

In [43]:
# access claudette messages
c.content[0].text

'----- ANSWER EVAL -----\n\n**GOOD**:\n1. Provided a high-level overview of relevant work experience as a research analyst.\n2. Conveyed confidence in handling challenging situations.\n\n**BAD**:\n1. The response is quite vague and lacks specific details about the candidate\'s work experience.\n2. The language used is informal and does not reflect the professionalism expected for a senior-level position.\n3. The answer does not demonstrate the depth of knowledge and expertise required for a senior research analyst role.\n\n[RESPONSE METRICS]\n{\n  "Linguistic Clarity": 3.0,\n  "Confidence": 5.0,\n  "Technical Expertise": 2.0,\n  "Strategic Thinking": 2.0,\n  "Impact Demonstration": 2.0,\n  "Industry Understanding": 2.0\n}\n\n----- REWRITTEN -----\n\n**As a senior research analyst, I have extensive experience in conducting in-depth market research, data analysis, and providing strategic insights to support key business decisions.** During my time at [Previous Employer], I led a team of 

In [55]:
import gradio as gr

system_prompt = """You are a helpful and concise assistant."""
model = claudette.models[3]
print(f'\nChosen model: {model}')
chat = claudette.Chat(model, sp=system_prompt)

def respond(user_message, history):
    print("Messages received so far:", user_message)  # Print to see outside gradio
    c = chat(user_message)
    return c.content[0].text 

chatbot = gr.ChatInterface(
    fn=respond,
    examples=["Hello Neo.. let's begin."],
    theme="default"
)

chatbot.launch(inline=True) 


Chosen model: claude-3-haiku-20240307


d:\ProgramData\Anaconda3.9\envs\wingman\Lib\site-packages\gradio\components\chatbot.py:291: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7876

To create a public link, set `share=True` in `launch()`.


Messages received so far: Hello Neo.. let's begin.


----- ANSWER EVAL -----

**GOOD**:
1. Provided a high-level overview of relevant work experience as a research analyst.
2. Conveyed confidence in handling challenging situations.

**BAD**:
1. The response is quite vague and lacks specific details about the candidate's work experience.
2. The language used is informal and does not reflect the professionalism expected for a senior-level position.
3. The answer does not demonstrate the depth of knowledge and expertise required for a senior research analyst role.

[RESPONSE METRICS]
{
  "Linguistic Clarity": 3.0,
  "Confidence": 5.0,
  "Technical Expertise": 2.0,
  "Strategic Thinking": 2.0,
  "Impact Demonstration": 2.0,
  "Industry Understanding": 2.0
}

----- REWRITTEN -----

**As a senior research analyst, I have extensive experience in conducting in-depth market research, data analysis, and providing strategic insights to support key business decisions.** During my time at [Previous Employer], I led a team of analysts in a comprehensive study of the competitive landscape within our industry. This involved gathering and synthesizing data from multiple sources, including industry reports, customer surveys, and internal sales data. 

**I was able to identify emerging trends and pain points that our clients were facing, which allowed us to develop a targeted product strategy that ultimately drove a 15% increase in revenue over the course of two years.** In addition, I have a proven track record of presenting complex research findings to both technical and non-technical stakeholders in a clear and compelling manner. I am highly proficient in using a variety of data analysis tools and techniques, such as SQL, Python, and Tableau, to ensure the accuracy and reliability of my research.

**Throughout my career, I have consistently demonstrated the ability to juggle multiple projects simultaneously while maintaining a keen eye for detail and a focus on delivering high-quality, actionable insights.** I am passionate about staying up-to-date with the latest industry trends and developments, and I actively participate in professional associations and attend relevant conferences to expand my knowledge and network.

<details>

- id: `msg_01QuqGjPid8zJDx4tGh6tYNe`
- content: `[{'citations': None, 'text': '----- ANSWER EVAL -----\n\n**GOOD**:\n1. Provided a high-level overview of relevant work experience as a research analyst.\n2. Conveyed confidence in handling challenging situations.\n\n**BAD**:\n1. The response is quite vague and lacks specific details about the candidate\'s work experience.\n2. The language used is informal and does not reflect the professionalism expected for a senior-level position.\n3. The answer does not demonstrate the depth of knowledge and expertise required for a senior research analyst role.\n\n[RESPONSE METRICS]\n{\n  "Linguistic Clarity": 3.0,\n  "Confidence": 5.0,\n  "Technical Expertise": 2.0,\n  "Strategic Thinking": 2.0,\n  "Impact Demonstration": 2.0,\n  "Industry Understanding": 2.0\n}\n\n----- REWRITTEN -----\n\n**As a senior research analyst, I have extensive experience in conducting in-depth market research, data analysis, and providing strategic insights to support key business decisions.** During my time at [Previous Employer], I led a team of analysts in a comprehensive study of the competitive landscape within our industry. This involved gathering and synthesizing data from multiple sources, including industry reports, customer surveys, and internal sales data. \n\n**I was able to identify emerging trends and pain points that our clients were facing, which allowed us to develop a targeted product strategy that ultimately drove a 15% increase in revenue over the course of two years.** In addition, I have a proven track record of presenting complex research findings to both technical and non-technical stakeholders in a clear and compelling manner. I am highly proficient in using a variety of data analysis tools and techniques, such as SQL, Python, and Tableau, to ensure the accuracy and reliability of my research.\n\n**Throughout my career, I have consistently demonstrated the ability to juggle multiple projects simultaneously while maintaining a keen eye for detail and a focus on delivering high-quality, actionable insights.** I am passionate about staying up-to-date with the latest industry trends and developments, and I actively participate in professional associations and attend relevant conferences to expand my knowledge and network.', 'type': 'text'}]`
- model: `claude-3-haiku-20240307`
- role: `assistant`
- stop_reason: `end_turn`
- stop_sequence: `None`
- type: `message`
- usage: `{'cache_creation_input_tokens': 0, 'cache_read_input_tokens': 0, 'input_tokens': 816, 'output_tokens': 480}`

</details>

In [54]:
#| hide
import nbdev; nbdev.nbdev_export()